In [ ]:
# R     ==> RANDOM  +   NO CLASS    (No class)          (RANDOM)
# C     ==> RANDOM  +   STRATIFIED  (class from latent) (RANDOM)
# CF    ==> FPS     +   STRATIFIED  (class from latent) (FPS Distance from 2D)
# CFW   ==> FPS     +   NO CLASS    (No class)          (FPS Distance from 2D)

# CFWX  ==> FPS     +   NO CLASS    (No class)          (FPS Distance from latent)
# CFX   ==> FPS     +   STRATIFIED  (class from latent) (FPS Distance from latent)

# RANDOM - FPS
# STRATIFIED - WHOLE (NO CLASS)


# ALL   ==> COMPLESE CAS DATASET
# SOTA  ==> (CF) DIVERSITY SAMPLING OVER SOTA CLASS

In [ ]:
import glob

def find_files(path, extension='txt', word='results'):
    path = path[:-1] if path.endswith('/') else path
    search_pattern = f'{path}/**/*{word}*.{extension}'
    files = glob.glob(search_pattern, recursive=True)
    relative_files = [file.replace(f"{path}/", '') for file in files]
    return relative_files

In [ ]:
final_exp_path = '/data/falcetta/brain_data/FINAL_EXP_SEEDs'
final_exp_path = '/data/falcetta/brain_data/FINAL_EXP_VDISNET_JOURNAL_SEEDs'
word = 'results_NO_PNET_50_OVERLAP' # 'results_NO_PNET_50_OVERLAP' 'results_PNET_50_OVERLAP'
plot_std = True
metric ='DICE'


In [ ]:
import re
extension = 'txt'
files = find_files(final_exp_path, extension, word)

for file in files:
    print(file)

In [ ]:
plot_dict_dc = {}
plot_dict_cldc = {}
error_models = []

for file in files:
    print(file)
    # Open the file and read the content
    model = file.split('/')[1]
    n_size = file.split('/')[0] if 'TOT' not in file.split('/')[0] else 'TOT'
    
    if model == 'CFWX':
        continue
    
    if model not in plot_dict_cldc:
        plot_dict_cldc[model] = [[],[],[]] #  N_SIZE, AVG, STD
        plot_dict_dc[model] = [[],[],[]] #  N_SIZE, AVG, STD
    
    with open(f"{final_exp_path}/{file}", 'r') as f:
        content = f.read()
        #print(content)
        # Extract AVG and STD for DICE
        dice_avg = re.search(r'\|  AVG \|   (\d+\.\d+) \|', content)
        dice_std = re.search(r'\|  STD \|   (\d+\.\d+) \|', content)

        # Extract AVG and STD for CLDICE
        cldice_avg = re.search(r'\|  AVG \|   \d+\.\d+ \|   (\d+\.\d+) \|', content)
        cldice_std = re.search(r'\|  STD \|   \d+\.\d+ \|   (\d+\.\d+) \|', content)

        if dice_avg is None or dice_std is None or cldice_avg is None or cldice_std is None:
            print(f"ERROR: {model} - {n_size}")
            error_models.append(file)
            continue
        
        if float(dice_avg.group(1))<0.1 or float(cldice_avg.group(1))<0.1:
            print(f"ERROR (F): {model} - {n_size}")
            error_models.append(file)
            continue        
        # Print the extracted values
        print(f"MODEL {model} - N_SIZE {n_size}")
        print("DICE AVG:", dice_avg.group(1))
        print("DICE STD:", dice_std.group(1))
        print("CLDICE AVG:", cldice_avg.group(1))
        print("CLDICE STD:", cldice_std.group(1))

        # Save the extracted values
        plot_dict_dc[model][0].append(n_size)
        plot_dict_dc[model][1].append(float(dice_avg.group(1)))
        plot_dict_dc[model][2].append(float(dice_std.group(1)))
        
        plot_dict_cldc[model][0].append(n_size)
        plot_dict_cldc[model][1].append(float(cldice_avg.group(1)))
        plot_dict_cldc[model][2].append(float(cldice_std.group(1)))

In [ ]:
print("MODEL WITH ERRORS (YOU NEED TO RE-RUN)")
for err_model in error_models:
    print(err_model)


In [ ]:
print(plot_dict_dc)
if metric =='CLDICE':
    data = plot_dict_cldc
else:
    data = plot_dict_dc
# Iterate over the items in the dictionary
for key, value in data.items():
    print(key, value)
    if key == 'all':
        continue
    # Sort the lists based on the first list
    sorted_lists = sorted(zip(*value), key=lambda x: float(x[0]))
    # Unzip the sorted lists
    sorted_lists = list(zip(*sorted_lists))
    # Update the value in the dictionary
    data[key] = sorted_lists

print(data)

for model, values in data.items():
    print(f"\n------ MODEL {model} ------")
    print("N_SIZE:", values[0])
    print("DICE AVG:", values[1])
    print("DICE STD:", values[2])

In [ ]:
import numpy as np 
result = {}

for key, value in data.items():
    print(f"KEY: {key}")
    ids = value[0]
    dice = value[1]
    
    result[key] = [[],[],[]]
    
    # aggregate the same id and calculate the mean and std
    mean_dict = {}
    std_dict = {}
    for i, id in enumerate(ids):
        if id not in mean_dict:
            mean_dict[id] = []
            std_dict[id] = []
        mean_dict[id].append(dice[i])
        std_dict[id].append(dice[i])
    
    print(mean_dict)
    print(std_dict)
    
    for id, values in mean_dict.items():
        mean = np.mean(values)
        std = np.std(values)
        result[key][0].append(id)
        result[key][1].append(mean)
        result[key][2].append(std)


In [ ]:
for model, values in result.items():
    print(f"\n------ MODEL {model} ------")
    print("N_SIZE:", values[0])
    print("DICE AVG:", values[1])
    print("DICE STD:", values[2])

In [ ]:
data = result

In [ ]:
# do an horizontal line from 0 to 1
if 'all' in data:
    data['all'][0] = ['0.001', '0.01', '0.05', '0.1', '0.3' ,'0.5', '0.75']
    data['all'][1] = data[model][1]*len(data[model][0])
    data['all'][2] = data[model][2]*len(data[model][0])

#change key name from all to Upper Bound
data['R-C'] = data.pop('C') # RANDOM CLASS 
data['FT-C'] = data.pop('CF') # FPS CLASS TSNE
data['FL-C'] = data.pop('CFX') # FPS CLASS LATENT
data['R-W'] = data.pop('R') # RANDOM WHOLE
#data['FT-W'] = data.pop('CFW') # FPS WHOLE TSNE
#data['SOTA-R-C'] = data.pop('SOTAC') # SOTA RANDOM CLASS
data['Full Dataset'] = data.pop('all')
#data['RA'] = data.pop('YYY') # SOTA FPS CLASS TSNE
data['CA'] = data.pop('SOTACF') # SOTA FPS CLASS TSNE
#data['AET'] = data.pop('XXXX') # SOTA FPS CLASS TSNE


#'R-W'     RANDOM WHOLE
#'R-C'     RANDOM CLASS 
#'FT-C'    FPS TSNE CLASS 
#'FT-W'    FPS WHOLE TSNE
#'FL-C'    FPS LATENT CLASS 
#'SOTA-R-C' SOTA RANDOM CLASS
#'SOTA-FT-C' SOTA FPS CLASS TSNE

In [ ]:
print("DIFFERENT MODELS:")
for i, (model, values) in enumerate(data.items()):
    print(f"{i+1}: {model}")

print("\nSORTED:")
for i, (model, values) in enumerate(sorted(data.items())):
    print(f"{i+1}: {model}")


In [ ]:
def get_color_name(color_code):
    # Define the color mappings
    dark_colors = {
        '#ff7f0e': 'Vivid Orange',
        '#2ca02c': 'Forest Green',
        '#9467bd': 'Amethyst',
        '#1f77b4': 'Deep Sky Blue',
        '#7f7f7f': 'Gray',
        '#dc143c': 'Crimson',
        '#e377c2': 'Pale Magenta',
        '#008080': 'Teal'
    }

    light_colors = {
        '#ffbb78': 'Light Orange',
        '#98df8a': 'Light Green',
        '#c5b0d5': 'Light Purple',
        '#aec7e8': 'Light Sky Blue',
        '#c7c7c7': 'Light Gray',
        '#f08080': 'Light Coral',
        '#f7b6d2': 'Light Pink',
        '#20b2aa': 'Light Teal'
    }

    # Check and return the name of the color
    if color_code in dark_colors:
        return dark_colors[color_code]
    elif color_code in light_colors:
        return light_colors[color_code]
    else:
        return 'Unknown Color'

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_with_std(averages, stds, model ,x_values ,color='blue', light_color='lightblue', plot_std=True):
    # Create the line plot
    x_values = [f'{int(100*float(x))}' for x in x_values]
    x_values[0] = '0.1'
    plt.plot(x_values, averages, '-o', color=color, label=model)

    if plot_std:
        # Fill the area around the average line with standard deviations
        plt.fill_between(x_values, np.subtract(averages, stds), np.add(averages, stds),
                        color=light_color, alpha=0.3)

        
plt.figure(figsize=(15, 6))

# Reorganized Dark Colors
colors = ['#ff7f0e', '#2ca02c', '#9467bd', '#1f77b4', '#7f7f7f', '#dc143c', '#e377c2', '#008080']

# Reorganized Light Colors
light_colors = ['#ffbb78', '#98df8a', '#c5b0d5', '#aec7e8', '#c7c7c7', '#f08080', '#f7b6d2', '#20b2aa']
                #  Orange,   Green  ,   Purple ,   Blue   ,   Gray   ,   Coral  ,    Pink  , Teal

for i, (model, values) in enumerate(data.items()):
    #print(model, values)
    color = colors[i]
    light_color = light_colors[i]

    plot_with_std(values[1], values[2], model, x_values=values[0], color=color, light_color=light_color, plot_std=plot_std)

plt.xlabel('Labeled Data (%)', fontsize=14)
plt.ylabel('CLDice Score (%)' if metric == 'CLDICE' else 'Dice Score (%)', fontsize=14)
plt.title('CAS dataset', fontsize=17)
plt.legend(fontsize='xx-large')
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()





In [ ]:
plt.figure(figsize=(12, 6))

for i, (model, values) in enumerate(data.items()):
    #print(model, values)
    color = colors[i]
    light_color = light_colors[i]
    
    if 'SOTA' in model:
        continue

    plot_with_std(values[1], values[2], model, x_values=values[0], color=color, light_color=light_color, plot_std=plot_std)

plt.yticks([i/100 for i in range(25, 91, 5)])  # Set custom y-ticks
plt.ylim(0.25, 0.91)  # Adjust the y-axis limits
plt.xlabel('Labeled Data (%)', fontsize=14, fontname='Latin Modern Math')
plt.ylabel('CLDice Score (%)' if metric == 'CLDICE' else 'Dice Score (%)', fontsize=14, fontname='Latin Modern Math')
plt.title('CAS dataset', fontsize=17, fontname='Latin Modern Math')
plt.legend(fontsize='xx-large')
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()


In [ ]:
plt.figure(figsize=(10, 7))

for i, (model, values) in enumerate(data.items()):
    
    if model in [
                    #'R-C',
                    #'FT-C',
                    'FL-C',
                    'R-W',
                    'Full Dataset'
                    'RA',
                    'CA',
                    'AET'
                    ]:
        print(model, values)
        
        if model == 'FL-C':
            FLC_values = values
        
        
        color = colors[i]
        light_color = light_colors[i]
        
        plot_with_std(values[1], values[2], model, x_values=values[0], color=color, light_color=light_color, plot_std=plot_std)

#style latin modernmath
plt.xticks(fontsize='large')
plt.yticks(fontsize='large')
plt.xlabel('Labeled Data (%)', fontsize=25, fontname='Latin Modern Math')
plt.ylabel('CLDice Score (%)' if metric == 'CLDICE' else 'Dice Score (%)', fontsize=25, fontname='Latin Modern Math')
#plt.title('SOTA Comparison - CAS dataset', fontsize=17, fontname='Latin Modern Math')
plt.legend(fontsize='xx-large')
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()


In [ ]:
def compute_diff(values1, values2):
    # values1 and values2 are lists of the same length. each one contains 3 lists: N_SIZE, AVG, STD
    # compute the difference between the two values for each N_SIZE
    diff = [[],[],[]]
    for i in range(len(values1[0])):
        diff[0].append(values1[0][i])
        diff[1].append(values1[1][i] - values2[1][i])
        #diff[2].append(values1[2][i] + values2[2][i])
    avg_diff = np.mean(diff[1])
    diff = np.array(diff[1])
    return diff, avg_diff
    
    

In [ ]:
for i, (model, values) in enumerate(data.items()):
    
    if model in [
                    'CA',
                    #'R-C',
                    #'FT-C',
                    'R-W',
                    #FT-W',
                    'FL-C',
                    'Full Dataset']:
        #print(model, values)
        diff = []
        avg_diff = 0
        diff, avg_diff = compute_diff(FLC_values,values)
        print(f"Compare FL-C and {model}")
        print(f"Average Difference: {avg_diff*100}")
        print(f"Difference: {100*diff}")
        
        

In [ ]:
plt.figure(figsize=(10, 7))

for i, (model, values) in enumerate(data.items()):
    
    if model in [
                    #'SOTA-R-C',
                    #'SOTA-FT-C',
                    'R-C',
                    'FT-C',
                    #'R-W',
                    #'FT-W',
                    'FL-C',
                    'Full Dataset']:
        print(model, values)
        
        color = colors[i]
        light_color = light_colors[i]

        if model == 'Upper Bound':
            old_plot_std = plot_std
            plot_std = False
        plot_with_std(values[1], values[2], model, x_values=values[0], color=color, light_color=light_color, plot_std=plot_std)
        if model == 'Upper Bound':
            plot_std = old_plot_std
#style latin modernmath
plt.xticks(fontsize='large')
plt.yticks(fontsize='large')
plt.xlabel('Labeled Data (%)', fontsize=25, fontname='Latin Modern Math')
plt.ylabel('CLDice Score (%)' if metric == 'CLDICE' else 'Dice Score (%)', fontsize=25, fontname='Latin Modern Math')
#plt.title('Settings comparison (OUR) - CAS dataset', fontsize=17, fontname='Latin Modern Math')
plt.legend(fontsize='xx-large')
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_with_std2(averages, stds, model ,x_values ,color='blue', light_color='lightblue', plot_std=True):
    # Create the line plot
    #remove first and last value
    x_values = x_values[1:-1]
    averages = averages[1:-1]
    stds = stds[1:-1]
    
    x_values = [f'{100*float(x)}' for x in x_values]
    plt.plot(x_values, averages, '-o', color=color, label=model)

    if plot_std:
        # Fill the area around the average line with standard deviations
        plt.fill_between(x_values, np.subtract(averages, stds), np.add(averages, stds),
                        color=light_color, alpha=0.3)

        
plt.figure(figsize=(15, 6))


for i, (model, values) in enumerate(data.items()):
    #print(model, values)
    color = colors[i]
    light_color = light_colors[i]

    plot_with_std2(values[1], values[2], model, x_values=values[0], color=color, light_color=light_color, plot_std=plot_std)

plt.xlabel('Labeled Data (%)', fontsize=14)
plt.ylabel('CLDice Score (%)' if metric == 'CLDICE' else 'Dice Score (%)', fontsize=14)
plt.title('CAS dataset', fontsize=17)
plt.legend(fontsize='xx-large')
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

In [ ]:
for i, (model, values) in enumerate(data.items()):
    #print(model, values)
    plt.figure(figsize=(15, 6))
    color = colors[i]
    light_color = light_colors[i]

    # find color name
    print(model)
    print(get_color_name(color), get_color_name(light_color))
    print()
    plot_with_std(values[1], values[2], model, x_values=values[0], color=color, light_color=light_color, plot_std=plot_std)

    plt.xlabel('Labeled Data (%)', fontsize=14)
    plt.ylabel('CLDice Score (%)' if metric == 'CLDICE' else 'Dice Score (%)', fontsize=14)
    plt.title('CAS dataset', fontsize=17)
    plt.legend(fontsize='xx-large')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()